# Dicey

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import requests
import re
import time
import pickle
pd.options.display.max_seq_items = 2000

In [3]:
# Set up dataset
bgg = pd.read_csv("./Cleaned_BGG")
bgg.drop('Unnamed: 0',axis = 1, inplace = True)
bgg.drop_duplicates(subset= 'names', inplace= True)
bgg['names'].value_counts().head()

Duel in the Dark                           1
Flowerpower                                1
...and then, we held hands.                1
Advanced European Theater of Operations    1
Gettysburg: Badges of Courage              1
Name: names, dtype: int64

## Data Collection

In [4]:
# # #Collect User list
# url = "https://boardgamegeek.com/findgamers.php?action=findclosest&country=US&srczip=07057&maxdist=100&B1=Submit"
# response = requests.get(url)
# html = response.text

# soup = BeautifulSoup(html, 'lxml')

# hrefs = []
# for link in soup.find_all('a'):
#     hrefs.append(link.get('href'))
    
# # #Remove Nonetypes
# hrefs = filter(None.__ne__, hrefs)
# hrefs = list(hrefs)

# # # Grab Usernames
# usernames = []
# for _ in hrefs:
#     if re.search('\/user\/', _):
#         _ = _[6:]
#         usernames.append(_)
# usernames = list(set(usernames))

In [5]:
# pickle.dump(usernames, open("large_un.p", "wb"))

In [6]:
# user_choices = {}

In [7]:
# # Collect User Data
# for user in usernames:
#     games = []
#     res = requests.get('http://www.boardgamegeek.com/xmlapi/collection/'+user+'?minrating=8')
#     soup = BeautifulSoup(res.text, "lxml")
#     for link in soup.find_all('name'):
#         game = re.findall("(?<=name sortindex=\"1\">).*(?=</name)", str(link))
#         games.extend(game)
#         user_choices.update({user:games})
#         time.sleep(2)

In [8]:
# #Save user choices
# pickle.dump(user_choices, open("list.p", "wb"))

## Cleanup

In [9]:
# Load user choices
user_choices= pickle.load(open("gamelist.p", "rb"))

In [10]:
user_names= pickle.load(open("usernames.p", "rb"))

In [11]:
testlist = []

for name in user_choices.keys():
    for game in user_choices[name]:
        testlist.append(name+'~'+game)

In [12]:
df_user= []
df_game = []
for _ in testlist:
    user, game= _.split('~')
    df_user.append(user)
    df_game.append(game)

## Merge Data

In [13]:
# Create User listing
rec_user = pd.DataFrame({'User':df_user, 'Game': df_game})
rec_user = rec_user.reindex(columns = ['User','Game'])
rec_user.head()

,User,Game
0,MSRyanInNYC,Agents of SMERSH
1,MSRyanInNYC,Batailles de L'Ancien Regime 1740-1763
2,MSRyanInNYC,Battle Cry: 150th Civil War Anniversary Edition
3,MSRyanInNYC,Battlegroup
4,MSRyanInNYC,Battlegroup Overlord


In [14]:
# # Safety Test
# test = rec_user.copy()
# bggtest = bgg.copy()

In [15]:
#Combine user data with games from main bgg dataframe
comp = pd.merge(rec_user, bgg, how= 'inner', left_on= 'Game', right_on='names')

## Algorithm creation

In [16]:
ui = "Pandemic"

In [17]:
# Set up df index
alg_rec = pd.DataFrame([])
alg_rec['name'] = bgg['names']
alg_rec = alg_rec.set_index('name')

In [18]:
# User Score
users = comp[comp['Game'] == ui ]
users = list(users['User'])
high_ranked= []

for _ in users:
    temp = comp[comp['User'] == _]
    high_ranked.extend(temp.iloc[:, 1])
high_ranked = pd.Series(high_ranked)
high_ranked = high_ranked.value_counts()
high_ranked = high_ranked * 0.04545453
alg_rec['user_score'] = high_ranked
alg_rec = alg_rec.fillna(0)

In [19]:
# Category Score
cat = pd.DataFrame(bgg.iloc[:,3])
cat = cat.join(bgg.iloc[:,69:])
cat.iloc[215:225,:]
target= cat[cat['names'] == ui]
cat_scores = []

for _ in cat['names']:
    test = cat[cat['names'] == _]
    temp = cosine_similarity(target.drop('names', axis = 1), test.drop('names', axis = 1))[0]
    cat_scores.extend(temp)

alg_rec['cat_scores'] = cat_scores

In [20]:
# Mechanic Score
mech = pd.DataFrame(bgg.iloc[:,3])
mech = mech.join(bgg.iloc[:,20:69])

target= mech[mech['names'] == ui]
mech_scores = []

for _ in mech['names']:
    test = mech[mech['names'] == _]
    temp = cosine_similarity(target.drop('names', axis = 1), test.drop('names', axis = 1))[0]
    mech_scores.extend(temp)
    
alg_rec['mech_scores'] = mech_scores

In [32]:
# The Great Decider
alg_rec['mech_scores'] = alg_rec['mech_scores'] * 2.0
alg_rec['cat_scores'] = alg_rec['cat_scores'] * 1.5
alg_rec['total'] = alg_rec ['user_score'] + alg_rec['cat_scores'] + alg_rec['mech_scores']
rec = alg_rec.sort_values('total', ascending= False).head(20)
rec = list(rec.index)
rec[1:11]

['Pandemic Legacy: Season 1',
 'Pandemic: Reign of Cthulhu',
 'Pandemic Iberia',
 'Kings of Israel',
 'Defenders of the Realm',
 'Marvel Heroes',
 'Black Orchestra',
 'Mystery Express',
 'Pandemic: The Cure',
 'Train Heist']